In [1]:
import aif360
print("AIF360 version:", aif360.__version__)
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.algorithms.preprocessing import Reweighing
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness


# Construct the full path to the CSV files
csv_path_student_depression = os.path.join('datasets', 'Depression Student Dataset.csv')
csv_path_mental_health = os.path.join('datasets', 'Mental Health Dataset.csv')

AIF360 version: 0.6.1


pip install 'aif360[inFairness]'
/opt/homebrew/Caskroom/miniforge/base/envs/aif360v2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/aif360v2/lib/python3.9/site-packages/fairgbm/lib_lightgbm.so, 0x0006): tried: '/opt/homebrew/Caskroom/miniforge/base/envs/aif360v2/lib/python3.9/site-packages/fairgbm/lib_lightgbm.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/Caskroom/miniforge/base/envs/aif360v2/lib/python3.9/site-packages/fairgbm/lib_lightgbm.so' (no such file), '/opt/homebrew/Caskroom/miniforge/base/envs/aif360v2/lib/python3.9/site-packages/fairgbm/lib_lightgbm.so' (not a mach-o file)

Dataset 2: 
- Higher gender bias. Around 0.05 Statistical Parity Difference, indicates that women are about 5% points more likely to be diagnosed with depression than men. 
- Less Age bias. Around -0.04 Statistical Parity Difference, which means people at age 22 or above are about 4% points less likely to be diagnosed with depression compared to individuals with a age below 22.
- Additionally the men to female ratio in the dataset is around 0.32 which means that for every male datapoint there are 3.2 female datapoints, indicating a inbalance. 


In [ ]:
# Read the CSV file into a pandas DataFrame
#df_mental_health = pd.read_csv(csv_path_mental_health)
# List the columns you want to encode
df_student_depression = pd.read_csv(csv_path_student_depression)
categorical_cols_student_depression = ['Gender', 'Depression', 'Financial Stress', 'Family History of Mental Illness', 'Have you ever had suicidal thoughts ?']
#Drop some values for the moment to make the dataset numerical
df_student_depression = df_student_depression.drop(['Sleep Duration', 'Dietary Habits'], axis=1)
# Encode each column in a loop
for col in categorical_cols_student_depression:
    le = LabelEncoder()
    df_student_depression[col] = le.fit_transform(df_student_depression[col])
    # Display the mapping
    print(f"Mapping of {col} to integers:")
    for index, class_ in enumerate(le.classes_):
        print(f"{class_} -> {index}")
        
# Categorize the "Age" feature
df_student_depression['Age_Category'] = df_student_depression['Age'].apply(lambda x: 1 if x > 26 else 0)
#Drop some values for the moment to make the dataset numerical
df_student_depression = df_student_depression.drop(['Age'], axis=1)
# List the value types of the dataset
print("Value types of the dataset:")
print(df_student_depression.dtypes)

# Display the first few rows of the DataFrame
print("Student Depression Dataset:")
print(df_student_depression.head())
#print("\nMental Health Dataset:")
#print(df_mental_health.head())



Mapping of Gender to integers:
Female -> 0
Male -> 1
Mapping of Depression to integers:
No -> 0
Yes -> 1
Mapping of Financial Stress to integers:
1 -> 0
2 -> 1
3 -> 2
4 -> 3
5 -> 4
Mapping of Family History of Mental Illness to integers:
No -> 0
Yes -> 1
Mapping of Have you ever had suicidal thoughts ? to integers:
No -> 0
Yes -> 1
Value types of the dataset:
Gender                                     int64
Academic Pressure                        float64
Study Satisfaction                       float64
Have you ever had suicidal thoughts ?      int64
Study Hours                                int64
Financial Stress                           int64
Family History of Mental Illness           int64
Depression                                 int64
Age_Category                               int64
dtype: object
Student Depression Dataset:
   Gender  Academic Pressure  Study Satisfaction  \
0       1                2.0                 4.0   
1       1                4.0                 5.0   

In [ ]:
# Read the CSV file into a pandas DataFrame
#df_mental_health = pd.read_csv(csv_path_mental_health)
# List the columns you want to encode
df_meantal_health = pd.read_csv(csv_path_mental_health)
df_meantal_health['YearOfStudy'] = df_meantal_health['YearOfStudy'].astype(str).str.lower()
df_meantal_health['Course'] = df_meantal_health['Course'].astype(str).str.lower()
categorical_cols_mental_health = ['Gender', 'Course', 'YearOfStudy']
#Drop some values for the moment to make the dataset numerical
df_meantal_health = df_meantal_health.drop(['Timestamp'], axis=1)
# Encode each column in a loop
for col in categorical_cols_mental_health:
    le = LabelEncoder()
    df_meantal_health[col] = le.fit_transform(df_meantal_health[col])
    # Display the mapping
    print(f"Mapping of {col} to integers:")
    for index, class_ in enumerate(le.classes_):
        print(f"{class_} -> {index}")
        
# Categorize the "Age" feature
df_meantal_health['Age_Category'] = df_meantal_health['Age'].apply(lambda x: 1 if x > 22 else 0)
#Drop some values for the moment to make the dataset numerical
#df_meantal_health = df_meantal_health.drop(['Age'], axis=1)
print(df_meantal_health['Age'].unique())

# List the value types of the dataset
print("Value types of the dataset:")
print(df_meantal_health.dtypes)

# Display the first few rows of the DataFrame
print("Student Depression Dataset:")
print(df_meantal_health.head())
#print("\nMental Health Dataset:")
#print(df_mental_health.head())

Mapping of Gender to integers:
Female -> 0
Male -> 1
Mapping of Course to integers:
accounting  -> 0
ala -> 1
banking studies -> 2
bcs -> 3
benl -> 4
biomedical science -> 5
biotechnology -> 6
bit -> 7
business administration -> 8
communication  -> 9
cts -> 10
diploma nursing -> 11
diploma tesl -> 12
econs -> 13
engin -> 14
engine -> 15
engineering -> 16
enm -> 17
fiqh -> 18
fiqh fatwa  -> 19
human resources -> 20
human sciences  -> 21
irkhs -> 22
islamic education -> 23
it -> 24
kenms -> 25
kirkhs -> 26
koe -> 27
kop -> 28
law -> 29
laws -> 30
malcom -> 31
marine science -> 32
mathemathics -> 33
mhsc -> 34
nursing  -> 35
pendidikan islam -> 36
pendidikan islam  -> 37
psychology -> 38
radiography -> 39
taasl -> 40
usuluddin  -> 41
Mapping of YearOfStudy to integers:
year 1 -> 0
year 2 -> 1
year 3 -> 2
year 4 -> 3
[24 18 25 20 19 21 23 22]
Value types of the dataset:
Gender                          int64
Age                             int64
Course                          int64
YearOfS

Dataset 1: 
- Little gender bias. Around -0.03 Statistical Parity Difference, indicates that women are about 3% points less likely to be diagnosed with depression than men.  
- Substancial Age bias. Around -0.18 Statistical Parity Difference, which means people at age 26 or above are about 18% points less likely to be diagnosed with depression compared to individuals with a age below 26.

In [ ]:

dataset_student_depression_gender = BinaryLabelDataset(
    df=df_student_depression,
    label_names=['Depression'],
    protected_attribute_names=['Gender']
)
privileged_groups_gender = [{'Gender': 1}]
unprivileged_groups_gender = [{'Gender': 0}]

dataset_student_depression_age = BinaryLabelDataset(
    df=df_student_depression,
    label_names=['Depression'],
    protected_attribute_names=['Age_Category']
)
privileged_groups_age = [{'Age_Category': 0}]
unprivileged_groups_age = [{'Age_Category': 1}]

# Create a metric object using your dataset and group definitions
metric_depression_gender = BinaryLabelDatasetMetric(
    dataset_student_depression_gender,
    privileged_groups=privileged_groups_gender,
    unprivileged_groups=unprivileged_groups_gender
)

# Compute bias metrics
stat_parity_diff_gender = metric_depression_gender.mean_difference()   # Statistical Parity Difference
disparate_impact_gender = metric_depression_gender.disparate_impact()    # Disparate Impact

print("Statistical Parity Difference (gender):", stat_parity_diff_gender)
print("Disparate Impact (gender):", disparate_impact_gender)
# Calculate the ratio of males to females
male_count = (df_student_depression['Gender'] == 1).sum()
female_count = (df_student_depression['Gender'] == 0).sum()
male_to_female_ratio = male_count / female_count if female_count != 0 else None

print("Male to Female Ratio:", male_to_female_ratio)

# Create a metric object using your dataset and group definitions
metric_depression_age = BinaryLabelDatasetMetric(
    dataset_student_depression_age,
    privileged_groups=privileged_groups_age,
    unprivileged_groups=unprivileged_groups_age
)

# Compute bias metrics
stat_parity_diff_age = metric_depression_age.mean_difference()   # Statistical Parity Difference
disparate_impact_age = metric_depression_age.disparate_impact()    # Disparate Impact

print("Statistical Parity Difference (age):", stat_parity_diff_age)
print("Disparate Impact (age):", disparate_impact_age)
over_26_count = (df_student_depression['Age_Category'] == 0).sum()
under_26_count = (df_student_depression['Age_Category'] == 1).sum()
over_to_under_26_ratio = over_26_count / under_26_count
#Age in Dataset Varies from 18 to 30 years
print("Over 26 to Under 26 Ratio:", over_to_under_26_ratio)

Statistical Parity Difference (gender): -0.03174754960554632
Disparate Impact (gender): 0.9385753931544865
Male to Female Ratio: 1.1361702127659574
Statistical Parity Difference (age): -0.18326693227091634
Disparate Impact (age): 0.6912751677852349
Over 26 to Under 26 Ratio: 1.0


Dataset 2: 
- Higher gender bias. Around 0.05 Statistical Parity Difference, indicates that women are about 5% points more likely to be diagnosed with depression than men. 
- Less Age bias. Around -0.04 Statistical Parity Difference, which means people at age 22 or above are about 4% points less likely to be diagnosed with depression compared to individuals with a age below 22.
- Additionally the men to female ratio in the dataset is around 0.32 which means that for every male datapoint there are 3.2 female datapoints, indicating a inbalance. 


In [ ]:

#print(df_meantal_health['Gender'].unique()) 
dataset_mental_health_gender = BinaryLabelDataset(
    df=df_meantal_health,
    label_names=['Depression'],
    protected_attribute_names=['Gender'], 
)
privileged_groups = [{'Gender': 1}] #Men
unprivileged_groups = [{'Gender': 0}] #Woman

dataset_mental_health_age = BinaryLabelDataset(
    df=df_meantal_health,
    label_names=['Depression'],
    protected_attribute_names=['Age_Category']
)
privileged_groups_age = [{'Age_Category': 0}]
unprivileged_groups_age = [{'Age_Category': 1}]

# Create a metric object using your dataset and group definitions
metric_mental_health_gender = BinaryLabelDatasetMetric(
    dataset_mental_health_gender,
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups
)

# Compute bias metrics
stat_parity_diff = metric_mental_health_gender.mean_difference()   # Statistical Parity Difference
disparate_impact = metric_mental_health_gender.disparate_impact()    # Disparate Impact
#ratio = metric_mental_health.ratio()

print("Statistical Parity Difference (gender):", stat_parity_diff)
print("Disparate Impact (gender):", disparate_impact)

# Calculate the ratio of males to females
male_count = (df_meantal_health['Gender'] == 1).sum()
female_count = (df_meantal_health['Gender'] == 0).sum()
male_to_female_ratio = male_count / female_count if female_count != 0 else None

print("Male to Female Ratio:", male_to_female_ratio)

# Create a metric object using your dataset and group definitions
metric_mental_health_age = BinaryLabelDatasetMetric(
    dataset_mental_health_age,
    privileged_groups=privileged_groups_age,
    unprivileged_groups=unprivileged_groups_age
)

# Compute bias metrics
stat_parity_diff = metric_mental_health_age.mean_difference()   # Statistical Parity Difference
disparate_impact = metric_mental_health_age.disparate_impact()    # Disparate Impact
#ratio = metric_mental_health.ratio()

print("Statistical Parity Difference (age):", stat_parity_diff)
print("Disparate Impact (age):", disparate_impact)


over_26_count = (df_meantal_health['Age_Category'] == 0).sum()
under_26_count = (df_meantal_health['Age_Category'] == 1).sum()
over_to_under_26_ratio = over_26_count / under_26_count
#Age in Dataset Varies from 18 to 25 years
print("Over 22 to Under 22 Ratio:", over_to_under_26_ratio)


Statistical Parity Difference (gender): 0.04890350877192978
Disparate Impact (gender): 1.1096901131333003
Male to Female Ratio: 0.3157894736842105
Statistical Parity Difference (age): -0.041598121836594726
Disparate Impact (age): 0.9166673690421004
Over 22 to Under 22 Ratio: 1.570694087403599


In [ ]:

#print(df_meantal_health['Gender'].unique()) 
dataset_mental_health_gender = BinaryLabelDataset(
    df=df_meantal_health,
    label_names=['Depression'],
    protected_attribute_names=['Gender'], 
)
privileged_groups = [{'Gender': 1}] #Men
unprivileged_groups = [{'Gender': 0}] #Woman

dataset_mental_health_age = BinaryLabelDataset(
    df=df_meantal_health,
    label_names=['Depression'],
    protected_attribute_names=['Age_Category']
)
privileged_groups_age = [{'Age_Category': 0}]
unprivileged_groups_age = [{'Age_Category': 1}]

# Create a metric object using your dataset and group definitions
metric_mental_health_gender = BinaryLabelDatasetMetric(
    dataset_mental_health_gender,
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups
)

# Compute bias metrics
stat_parity_diff = metric_mental_health_gender.mean_difference()   # Statistical Parity Difference
disparate_impact = metric_mental_health_gender.disparate_impact()    # Disparate Impact
#ratio = metric_mental_health.ratio()

print("Statistical Parity Difference (gender):", stat_parity_diff)
print("Disparate Impact (gender):", disparate_impact)

# Calculate the ratio of males to females
male_count = (df_meantal_health['Gender'] == 1).sum()
female_count = (df_meantal_health['Gender'] == 0).sum()
male_to_female_ratio = male_count / female_count if female_count != 0 else None

print("Male to Female Ratio:", male_to_female_ratio)

# Create a metric object using your dataset and group definitions
metric_mental_health_age = BinaryLabelDatasetMetric(
    dataset_mental_health_age,
    privileged_groups=privileged_groups_age,
    unprivileged_groups=unprivileged_groups_age
)

# Compute bias metrics
stat_parity_diff = metric_mental_health_age.mean_difference()   # Statistical Parity Difference
disparate_impact = metric_mental_health_age.disparate_impact()    # Disparate Impact
#ratio = metric_mental_health.ratio()

print("Statistical Parity Difference (age):", stat_parity_diff)
print("Disparate Impact (age):", disparate_impact)


over_26_count = (df_meantal_health['Age_Category'] == 0).sum()
under_26_count = (df_meantal_health['Age_Category'] == 1).sum()
over_to_under_26_ratio = over_26_count / under_26_count
#Age in Dataset Varies from 18 to 25 years
print("Over 22 to Under 22 Ratio:", over_to_under_26_ratio)


Statistical Parity Difference (gender): 0.04890350877192978
Disparate Impact (gender): 1.1096901131333003
Male to Female Ratio: 0.3157894736842105
Statistical Parity Difference (age): -0.041598121836594726
Disparate Impact (age): 0.9166673690421004
Over 22 to Under 22 Ratio: 1.570694087403599


We identifies two biases, age and gender, now we want to remove both using two different techniques 
For the dataset we will mitigate the age bias using AI 360 techniques by assigning weights to the classes. 

Approach 1: Assign weights to classes 

In [ ]:
# Assuming dataset_student_depression is your BinaryLabelDataset,
# and you've already defined privileged and unprivileged groups.
RW = Reweighing(unprivileged_groups=unprivileged_groups_age, privileged_groups=privileged_groups_age)
dataset_student_depression_age_transf = RW.fit_transform(dataset_student_depression_age)

#Check the assigned weights
print("Transformed Dataset (Age Bias Mitigated):")
print(" Weights for privileged (Age < 26):", dataset_student_depression_age_transf.instance_weights[df_student_depression['Age_Category'] == 0][:1])
print(" Weights for unprivileged (Age >= 26):", dataset_student_depression_age_transf.instance_weights[df_student_depression['Age_Category'] == 1][:1])

# For demonstration purposes, let's calculate the mean weight for each group
# To see if they are approximately equal
male_weights = dataset_student_depression_age_transf.instance_weights[df_student_depression['Age_Category'] == 1]
female_weights = dataset_student_depression_age_transf.instance_weights[df_student_depression['Age_Category'] == 0]

print("Mean weight for privileged (Age < 26):", np.mean(male_weights))
print("Mean weight for unprivileged (Age >= 26):", np.mean(female_weights))


Transformed Dataset (Age Bias Mitigated):
 Weights for privileged (Age < 26): [0.84563758]
 Weights for unprivileged (Age >= 26): [0.84459459]
Mean weight for privileged (Age < 26): 1.0000000000000002
Mean weight for unprivileged (Age >= 26): 1.0


Approach 2: Oversample the underrepresented group

In [ ]:
from imblearn.over_sampling import SMOTE

# Split the data by depression outcome.
df_dep = df_meantal_health[df_meantal_health['Depression'] == 1]
df_no_dep = df_meantal_health[df_meantal_health['Depression'] == 0]

# For each subgroup, determine the counts:
count_women_dep = len(df_dep[df_dep['Gender'] == 0])
count_men_dep   = len(df_dep[df_dep['Gender'] == 1])
count_women_no  = len(df_no_dep[df_no_dep['Gender'] == 0])
count_men_no    = len(df_no_dep[df_no_dep['Gender'] == 1])

# Oversample men in each outcome group using SMOTE if necessary.
def oversample_gender(df_subgroup, target_gender=1, majority_count=None):
    # Use Gender as target; features are all columns except Gender.
    X = df_subgroup.drop('Gender', axis=1)
    y = df_subgroup['Gender']
    
    # Only oversample if the minority group count is less than the desired majority_count.
    current_count = (y == target_gender).sum()
    if majority_count is None or current_count >= majority_count:
        return df_subgroup
    smote = SMOTE(sampling_strategy={target_gender: majority_count}, random_state=42)
    X_res, y_res = smote.fit_resample(X, y)
    df_res = X_res.copy()
    df_res['Gender'] = y_res
    return df_res

# Oversample within the depression group:
target_dep = max(count_women_dep, count_men_dep)
df_dep_bal = oversample_gender(df_dep, target_gender=1, majority_count=target_dep)
# For women in depression group, no oversampling is needed; they are already the majority if count_women_dep >= count_men_dep.
# (If not, you would adjust similarly, but here our goal is to balance men up to the majority count.)

# Oversample within the non-depression group:
target_no_dep = max(count_women_no, count_men_no)
df_no_dep_bal = oversample_gender(df_no_dep, target_gender=1, majority_count=target_no_dep)

# Combine the balanced subgroups.
df_balanced_strat = pd.concat([df_dep_bal, df_no_dep_bal]).reset_index(drop=True)
df_meantal_health = df_balanced_strat

# Check the counts by Gender and Depression:
print("Balanced counts by Gender and Depression:")
print(df_balanced_strat.groupby(['Depression', 'Gender']).size())

print("\nOverall Gender Counts:")
print(df_balanced_strat['Gender'].value_counts())

Balanced counts by Gender and Depression:
Depression  Gender
0           0         384
            1         384
1           0         376
            1         376
dtype: int64

Overall Gender Counts:
Gender
0    760
1    760
Name: count, dtype: int64


In [ ]:
#Redo the analysis for Dataset 2
#print(df_meantal_health['Gender'].unique()) 
dataset_mental_health_gender = BinaryLabelDataset(
    df=df_meantal_health,
    label_names=['Depression'],
    protected_attribute_names=['Gender'], 
)
privileged_groups = [{'Gender': 1}] #Men
unprivileged_groups = [{'Gender': 0}] #Woman

dataset_mental_health_age = BinaryLabelDataset(
    df=df_meantal_health,
    label_names=['Depression'],
    protected_attribute_names=['Age_Category']
)
privileged_groups_age = [{'Age_Category': 0}]
unprivileged_groups_age = [{'Age_Category': 1}]

# Create a metric object using your dataset and group definitions
metric_mental_health_gender = BinaryLabelDatasetMetric(
    dataset_mental_health_gender,
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups
)

# Compute bias metrics
stat_parity_diff = metric_mental_health_gender.mean_difference()   # Statistical Parity Difference
disparate_impact = metric_mental_health_gender.disparate_impact()    # Disparate Impact
#ratio = metric_mental_health.ratio()

print("Statistical Parity Difference (gender):", stat_parity_diff)
print("Disparate Impact (gender):", disparate_impact)

# Calculate the ratio of males to females
male_count = (df_meantal_health['Gender'] == 1).sum()
female_count = (df_meantal_health['Gender'] == 0).sum()
male_to_female_ratio = male_count / female_count if female_count != 0 else None

print("Male to Female Ratio:", male_to_female_ratio)

# Create a metric object using your dataset and group definitions
metric_mental_health_age = BinaryLabelDatasetMetric(
    dataset_mental_health_age,
    privileged_groups=privileged_groups_age,
    unprivileged_groups=unprivileged_groups_age
)

# Compute bias metrics
stat_parity_diff = metric_mental_health_age.mean_difference()   # Statistical Parity Difference
disparate_impact = metric_mental_health_age.disparate_impact()    # Disparate Impact
#ratio = metric_mental_health.ratio()

print("Statistical Parity Difference (age):", stat_parity_diff)
print("Disparate Impact (age):", disparate_impact)


over_26_count = (df_meantal_health['Age_Category'] == 0).sum()
under_26_count = (df_meantal_health['Age_Category'] == 1).sum()
over_to_under_26_ratio = over_26_count / under_26_count
#Age in Dataset Varies from 18 to 25 years
print("Over 22 to Under 22 Ratio:", over_to_under_26_ratio)


Statistical Parity Difference (gender): 0.0
Disparate Impact (gender): 1.0
Male to Female Ratio: 1.0
Statistical Parity Difference (age): -0.06383942948716714
Disparate Impact (age): 0.8763881317072806
Over 22 to Under 22 Ratio: 1.9400386847195359


Learnings -> Gender Bias is removed, however Age Bias is higher. Careful with sampling techniques 

In [ ]:
# Add a 'Country' column to each dataset
df_student_depression['Country'] = 0  # India
df_meantal_health['Country'] = 1  # Malaysia

# Select only the required columns from each dataset
df_student_depression_subset = df_student_depression[['Gender', 'Age_Category', 'Depression', 'Country']]
df_meantal_health_subset = df_meantal_health[['Gender', 'Age_Category', 'Depression', 'Country']]

# Combine the two dataframes
df_combined = pd.concat([df_student_depression_subset, df_meantal_health_subset], ignore_index=True)

# Display the combined dataframe
print("Combined DataFrame with Country column:")
print(df_combined.head())

Combined DataFrame with Country column:
   Gender  Age_Category  Depression  Country
0       1             1           0        0
1       1             1           0        0
2       1             0           1        0
3       1             0           0        0
4       0             1           0        0
